In [ ]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

# Load your dataset
df = pd.read_csv('Dataset/games.csv')

df = df[df['Release Date'] != 'releases on TBD']

df['Release Date'] = df['Release Date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))
df['Year'] = df['Release Date'].dt.year

def convert_to_int(value):
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1000)
    else:
        return int(value)

df['Plays'] = df['Plays'].apply(convert_to_int)
df = df[df['Year'] <= 2023]

# Calculate average ratings by year
avg_ratings_by_year = df.groupby(['Year', 'Title'])['Rating'].mean().reset_index()

# Calculate overall average ratings by year
overall_avg_ratings_by_year = df.groupby('Year')['Rating'].mean().reset_index()

# Create a Dash app
app = dash.Dash(__name__)

# Define game titles for dropdown menu
game_titles = df['Title'].unique()

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='game-dropdown',
        options=[{'label': title, 'value': title} for title in game_titles],
        placeholder='Select game titles...',
        multi=True  # Allow multiple selections
    ),
    dcc.Graph(id='game-sales-plot')
])

# Define callback to update the plot based on the selected game titles
@app.callback(
    Output('game-sales-plot', 'figure'),
    [Input('game-dropdown', 'value')]
)
def update_plot(selected_games):
    if selected_games:
        # Filter the dataset based on the selected game titles
        filtered_df = avg_ratings_by_year[avg_ratings_by_year['Title'].isin(selected_games)]
        # Create a line plot for the selected game titles' average ratings
        fig = go.Figure()
        for game_title in selected_games:
            game_data = filtered_df[filtered_df['Title'] == game_title]
            fig.add_trace(go.Scatter(x=game_data['Year'], y=game_data['Rating'], mode='lines+markers', name=f'{game_title}'))
        
        # Add a line for overall average ratings with fixed color
        fig.add_trace(go.Scatter(x=overall_avg_ratings_by_year['Year'], y=overall_avg_ratings_by_year['Rating'], mode='lines', name='Overall Average Rating', line=dict(color='black')))
        
        fig.update_layout(title='Ratings Comparison', xaxis_title='Year', yaxis_title='Average Rating')
        return fig
    else:
        return {}
    


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
